<a href="https://colab.research.google.com/github/lakshaygola/TensorFlow-And-DeepLearning/blob/main/TransferLearning_Word_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.set_project('transferlearning-word-embedding')
jovian.set_colab_id('1C2XHCbVR8oEnNkQ1DK3gGpLb_0DgnBD9')

     |████████████████████████████████| 71kB 4.9MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
project_name = 'transferlearning-word-embedding'

# TransferLearning With Word Embedding

We will be using pretrained model for this project of word embedding.

Pretraind Model which we are using in the notebook is GloVe

We will create the small dataset and try to apply transfer learning on that and calculate the accuracy of the model.

In [5]:
!pip install jovian --upgrade --quiet
!pip install opendatasets

In [6]:
# Importing some necessary libraries
import jovian
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [7]:
# Creating the sample dataset
doc = ['Well done', 
       'Great Effort', 
       'Good Work', 
       'Nice',
       'Excellent',
       'you can do better', 
       'have to submit before deadline', 
       'This can be done better',
       'Poor', 
       'Not good',
       'Poor effort', 
       'Its ok this time but improve',
       'ok ok']

# Creating the label for the data
label = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1]

# 1 - Good remark
# 0 - Bad remark
doc

['Well done',
 'Great Effort',
 'Good Work',
 'Nice',
 'Excellent',
 'you can do better',
 'have to submit before deadline',
 'This can be done better',
 'Poor',
 'Not good',
 'Poor effort',
 'Its ok this time but improve',
 'ok ok']

In [8]:
# Converting the label into array
label  = np.array(label)
label

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1])

In [9]:
# Creating the tokenizer (help us to determin the vocal size)
token = Tokenizer()
token.fit_on_texts(doc)
vocal_size = len(token.word_index) + 1
print(vocal_size)

27


In [10]:
# Converting the text into integer values
encoded_doc = token.texts_to_sequences(doc)
print(encoded_doc)
print(token.word_index)

[[9, 2], [10, 3], [4, 11], [12], [13], [14, 5, 15, 6], [16, 17, 18, 19, 20], [7, 5, 21, 2, 6], [8], [22, 4], [8, 3], [23, 1, 7, 24, 25, 26], [1, 1]]
{'ok': 1, 'done': 2, 'effort': 3, 'good': 4, 'can': 5, 'better': 6, 'this': 7, 'poor': 8, 'well': 9, 'great': 10, 'work': 11, 'nice': 12, 'excellent': 13, 'you': 14, 'do': 15, 'have': 16, 'to': 17, 'submit': 18, 'before': 19, 'deadline': 20, 'be': 21, 'not': 22, 'its': 23, 'time': 24, 'but': 25, 'improve': 26}


In [11]:
# Finding the maximum length of the sentence in the list of text
length = max([len(sens.split()) for sens in doc])
print(length)

6


In [12]:
# Making the length of each sentence equal
padding_doc = pad_sequences(encoded_doc, maxlen = length, padding = 'post')
print(padding_doc)

[[ 9  2  0  0  0  0]
 [10  3  0  0  0  0]
 [ 4 11  0  0  0  0]
 [12  0  0  0  0  0]
 [13  0  0  0  0  0]
 [14  5 15  6  0  0]
 [16 17 18 19 20  0]
 [ 7  5 21  2  6  0]
 [ 8  0  0  0  0  0]
 [22  4  0  0  0  0]
 [ 8  3  0  0  0  0]
 [23  1  7 24 25 26]
 [ 1  1  0  0  0  0]]


In [15]:
# Open the file of the model
embedded_weights = dict()
f = open('/content/drive/MyDrive/Models/glove.6B.zip (Unzipped Files)/glove.6B.100d.txt')

for line in f:
  values = line.split()
  word = values[0]
  coeff = np.asarray(values[1:], dtype = 'float32')
  embedded_weights[word] = coeff
f.close()
print(len(embedded_weights))

400000


In [18]:
# Creating the matrix with the weights of the GloVe
embedded_matrix = np.zeros((vocal_size, 100))
for word, value in token.word_index.items():
  embedded_vector = embedded_weights.get(word)
  if embedded_vector is not None:
    embedded_matrix[value] = embedded_vector
print(embedded_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.21295001  0.03707     1.01240003 ... -0.26216    -0.069477
  -0.070536  ]
 [-0.2978      0.31147    -0.14937    ... -0.22709    -0.029261
   0.4585    ]
 ...
 [-0.024221   -0.034855    0.35710001 ... -0.087568    0.25961
   0.050783  ]
 [-0.057078    0.39873999  0.68861002 ... -0.19357    -0.030606
   0.2397    ]
 [ 0.017839   -0.010497    0.022748   ... -0.50676     0.70486999
   0.27588001]]


In [24]:
# Now Creating the model
model = Sequential()
model.add(Embedding(vocal_size, 100, input_length= length, weights = [embedded_matrix], trainable = False))
model.add(Flatten())
model.add(Dense(1, 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 6, 100)            2700      
_________________________________________________________________
flatten_3 (Flatten)          (None, 600)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 601       
Total params: 3,301
Trainable params: 601
Non-trainable params: 2,700
_________________________________________________________________
None


In [ ]:
# Now lets make the prediction and see the accuracy of the model
model.fit(padding_doc, label, epochs = 45, verbose=1)

In [27]:
# Evaluating the model
loss, accuracy = model.evaluate(padding_doc, label)
print('\nLoss of the model: ', loss)
print('Accuracy of the model: ', accuracy*100)

1/1 [==============================] - 0s 18ms/step - loss: 0.3179 - accuracy: 1.0000

Loss of the model:  0.3178699314594269
Accuracy of the model:  100.0


In [30]:
jovian.commit(project = project_name)

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ··········
[jovian] Uploading colab notebook to Jovian...
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/lakshaygola/transferlearning-word-embedding


'https://jovian.ai/lakshaygola/transferlearning-word-embedding'